# DSA — Digital Signature Algorithm

DSA is a digital signature scheme based on discrete logarithm (similar in spirit to ElGamal). In DSA, Bob signs a message with his private key and Alice verifies the signature using Bob’s public key. An eavesdropper (Eve) can see the message and signature but cannot forge a valid signature without the private key.

---

## Communication Flow

```
         Bob                                    Alice                         Eve
        -----                                   ------                        ----
     [Message m]
         |
         |--- Signs m with his private key ---> (m, signature)
         |                                     /         \
         |                     Verifies signature using Public key
         |                                     \
     [Signature valid?]                          Observes m and signature
```

*If the signature verification passes, Alice is assured that the message came from Bob and was not altered.*

---

## How It Works

### Key Generation

- **Public Parameters:**  
  - **p:** A large prime  
  - **q:** A prime divisor of (p - 1)  
  - **g:** A generator of a subgroup of order q in ℤₚ*

- **Keys:**  
  - **Private Key (x):** A random integer in the range 1 to q-1  
  - **Public Key (y):** Computed as:  
    `y = g^x mod p`

### Signing Process (Performed by Bob)

1. **Hash the Message:**  
   Compute the message hash:  
   `h = H(m)`  
   (H is typically a secure hash function like SHA-256.)

2. **Choose a Fresh Random Number:**  
   Pick a random number `k` in `1..q-1` (must never be reused).

3. **Compute r:**  
   Calculate  
   `r = (g^k mod p) mod q`

4. **Compute s:**  
   Compute the modular inverse of k (denoted as k⁻¹) and then:  
   `s = (k⁻¹ * (h + x * r)) mod q`

5. **Signature:**  
   The signature is the pair:  
   `(r, s)`

### Verification Process (Performed by Alice)

1. **Hash the Received Message:**  
   Compute  
   `h = H(m)`

2. **Compute w:**  
   Find the inverse of s modulo q:  
   `w = s⁻¹ mod q`

3. **Compute u1 and u2:**  
   - `u1 = (h * w) mod q`  
   - `u2 = (r * w) mod q`

4. **Compute v:**  
   Calculate  
   `v = ((g^u1 * y^u2) mod p) mod q`

5. **Verify Signature:**  
   The signature is valid if and only if  
   `v == r`

---

## Parameters

- **p, q, g:** Public group parameters  
  - q divides p-1  
  - g is a generator of a subgroup of ℤₚ* of order q

- **x:** Private key (random integer, 1 ≤ x ≤ q-1)  
- **y:** Public key, where `y = g^x mod p`  
- **k:** Fresh random per signature (must be unique per signature)  
- **H:** A cryptographic hash function (e.g., SHA-256)

---

## Security Notes

- **Freshness of k:** Never reuse the random value k across signatures. Reusing k can reveal the private key x.
- **Parameter Sizes:** Use large, standardized parameters to ensure security. Modern systems often prefer alternatives like Ed25519 or ECDSA for better performance and safety.
- **Always Hash:** The message must be hashed before signing to prevent vulnerabilities.

---

## Where It’s Used

- **Standards & Protocols:**  
  Traditionally used in various cryptographic standards and electronic signature systems.
- **Modern Systems:**  
  While DSA is still in use, many modern applications prefer Ed25519 or ECDSA for increased speed and improved

# Demo — DSA (Toy, educational; small numbers)

This uses tiny primes so you can see the math. Not secure.
In real life, use a library (see second code block)

In [ ]:
import hashlib, secrets

# --- Tiny toy parameters (INSECURE) ---
# Choose primes with q | (p-1). Here: p = 809, q = 101, since 809-1 = 808 = 8*101
p = 809
q = 101
h = 2
g = pow(h, (p - 1) // q, p)   # generator of order q
if g == 1:
    # try another h if needed (unlikely here)
    h = 3
    g = pow(h, (p - 1) // q, p)
assert pow(g, q, p) == 1 and g != 1

# --- Key generation (Bob) ---
x = secrets.randbelow(q - 1) + 1           # private in [1..q-1]
y = pow(g, x, p)                           # public

print("Public params (p,q,g):", (p, q, g))
print("Public key y:", y)
print("Private key x:", x)

# --- Helpers ---
def inv_mod(a, m):
    # Extended Euclid
    t, new_t = 0, 1
    r, new_r = m, a % m
    while new_r != 0:
        qk = r // new_r
        t, new_t = new_t, t - qk * new_t
        r, new_r = new_r, r - qk * new_r
    if r != 1:
        raise ValueError("no inverse")
    return t % m

def H(msg: bytes) -> int:
    return int.from_bytes(hashlib.sha256(msg).digest(), "big") % q

# --- Sign (Bob) ---
m = b"DSA demo message"
hval = H(m)

while True:
    k = secrets.randbelow(q - 1) + 1
    r = pow(g, k, p) % q
    if r == 0:
        continue
    try:
        kinv = inv_mod(k, q)
    except ValueError:
        continue
    s = (kinv * (hval + x * r)) % q
    if s == 0:
        continue
    break

sig = (r, s)
print("Signature (r, s):", sig)

# --- Verify (Alice) ---
r, s = sig
if not (1 <= r < q and 1 <= s < q):
    ok = False
else:
    w = inv_mod(s, q)
    u1 = (H(m) * w) % q
    u2 = (r * w) % q
    v = (pow(g, u1, p) * pow(y, u2, p)) % p
    v = v % q
    ok = (v == r)

print("Valid signature?", ok)


Public params (p,q,g): (809, 101, 256)
Public key y: 700
Private key x: 32
Signature (r, s): (94, 68)
✅ Valid signature? True


# Demo — Real DSA Sign/Verify (with cryptography)

This is the right way: use a vetted library.
If you prefer modern curves, use Ed25519 (shown below too).

In [2]:
from cryptography.hazmat.primitives.asymmetric import dsa, ed25519
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat

message = b"DSA real-world example"

# --- DSA (FIPS) ---
private_key = dsa.generate_private_key(key_size=2048)
public_key = private_key.public_key()

# Sign using SHA-256
from cryptography.hazmat.primitives.asymmetric import utils
signature = private_key.sign(message, hashes.SHA256())

# Verify
public_key.verify(signature, message, hashes.SHA256())
print("DSA signature verified (SHA-256).")

# (Optional) show public key bytes
pub_bytes = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
print(pub_bytes.decode()[:120], "...")

# --- Modern alternative: Ed25519 ---
ed_priv = ed25519.Ed25519PrivateKey.generate()
ed_pub = ed_priv.public_key()
ed_sig = ed_priv.sign(message)
ed_pub.verify(ed_sig, message)
print("Ed25519 signature verified.")


DSA signature verified (SHA-256).
-----BEGIN PUBLIC KEY-----
MIIDRzCCAjoGByqGSM44BAEwggItAoIBAQDhAvO9lGje76u7aT1t+fATuLvBuXyx
9p5XUcZ6DSlpv0G0SwolugiE1IrK ...
Ed25519 signature verified.
